## Import libraries

In [1]:
import os
import pandas as pd
import numpy as np
import copy
import torch
import loss
from torch import optim
from metrics import eval_metrics, get_epoch_acc
from dataloader import DataLoader
from cross_val import CrossVal
from torchvision import transforms
from eval import eval
from config import ModelParameters
from PIL import Image
import cv2
import metrics
from metrics import eval_metrics
from metrics import get_epoch_acc

# Import available models, you can also explore other PyTorch models
from cracknet import cracknet, CrackNet
# from unet import UNet
# from segnet import SegNet, SegResNet

In [5]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DEVICE = "cpu"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [3]:
num_fault = []
num_horizon = []
num_seismic = []

for name in os.listdir('../data/aug_fault_mask'):
    if name == '.ipynb_checkpoints':
        continue
    code = name.replace("fault","").replace(".npy","")
    if os.path.isfile('../data/aug_horizon_mask/horizon{}.npy'.format(code)) and os.path.isfile('../data/aug_raw_seismic/seismic{}.png'.format(code)):
        num_fault.append(name)
        num_horizon.append('horizon{}.npy'.format(code))
        num_seismic.append('seismic{}.png'.format(code))
    

df = pd.DataFrame({
    'RAW_SEISMIC': [f"{x}" for x in num_seismic],
    'RAW_FAULT': [f"{x}" for x in num_fault],
    'RAW_HORIZON': [f"{x}" for x in num_horizon]
})

In [4]:
cv = CrossVal(df, 3)
dataloaders = cv
# dataset = {}
# dataset['train'] = LabelMe(data_folder=os.path.join(DIR,'train'), transform=data_transforms['train'],
#                                 img_size=(1024, 1024))
# dataset['val'] = LabelMe(data_folder=os.path.join(DIR,'val'), transform=data_transforms['val'],
#                                 img_size=(1024, 1024))
# dataloaders = {x: torch.utils.data.DataLoader(dataset[x], batch_size = BATCH_SIZE,
#                                             shuffle = True, num_workers = 8, 
#                                             drop_last = False)
#                                             for x in ['train', 'val']}
class_count = len(dataloaders[0]['train'].dataset.label)

In [3]:
PATH = "../shared/models/cracknet cutoff focal loss revised w augmentation part 3.pt.pt"
# PATH = "../shared/models/unet cutoff focal loss revised w augmentation.pt.pt"
# PATH = "../shared/models/unet cutoff focal loss revised w augmentation part 2.pt.pt"
model = torch.load(PATH, weights_only=False)

In [23]:
idx = 200
dl = DataLoader([],[],[], (512, 512))
img = []
img.append(dl.input_transforms(Image.open(os.path.join("../data/aug_raw_seismic", df.at[idx, "RAW_SEISMIC"]))))
img = torch.stack(img, dim=0)
mask = []
mask.append(torch.from_numpy(cv2.resize(np.load(os.path.join("../data/aug_fault_mask", df.at[idx, "RAW_FAULT"])).astype(np.uint8), dl.IMAGE_SIZE, cv2.INTER_AREA).astype(np.int64)))
mask = torch.stack(mask, dim=0)
model = model.to(DEVICE)
img = img.to(DEVICE)
mask = mask.to(DEVICE)
mask_pred = model(img)

In [33]:
eval_metrics(mask_pred, mask, class_count, ModelParameters.EVAL_METRIC)

(0.7102247476577759, 1)

In [34]:
inter, union = eval_metrics(mask_pred, mask, class_count, 'batch_intersection_union')

2


In [35]:
get_epoch_acc(inter, union, 'batch_intersection_union')

np.float64(0.8276767921331318)

In [27]:
mask_pred1 = mask_pred[0]
mask_pred1.shape

torch.Size([2, 512, 512])

In [28]:
_, predict1 = torch.max(mask_pred1.data, 0)
predict1_np = predict1.numpy()
inverted_fault_mask = (255 - predict1_np * 255).astype('uint8')
transform = transforms.Resize((512, 512))
img = Image.open(os.path.join("../data/aug_raw_seismic", df.at[idx, "RAW_SEISMIC"]))
img = np.asarray(transform(img))
# Convert binary image to a 3-channel image for overlay (BGR format)
fault_mask_bgr = cv2.cvtColor(inverted_fault_mask, cv2.COLOR_GRAY2BGR)

overlay = cv2.addWeighted(img, 0.5, fault_mask_bgr, 0.5, 0)

cv2.imwrite("overlay_img_fault.png", overlay)

True

In [6]:
code = '1000_122_959_0'
code = '1240_1240_398_0'
code = '1234_558_383_1'
code = '1024_482_757_1'
# code = '1024'
dl = DataLoader([],[],[], (512, 512))
img = []
img.append(dl.input_transforms(Image.open(f"../data/aug_raw_seismic_hasfault/seismic-{code}.png")))
img = torch.stack(img, dim=0)
mask = []
mask.append(torch.from_numpy(cv2.resize(np.load(f"../data/aug_fault_mask_filter_hasfault/fault-{code}.npy").astype(np.uint8), dl.IMAGE_SIZE, cv2.INTER_AREA).astype(np.int64)))
mask = torch.stack(mask, dim=0)
model = model.to(DEVICE)
img = img.to(DEVICE)
mask = mask.to(DEVICE)
mask_pred = model(img)

In [7]:
mask_pred1 = mask_pred[0]
mask_pred1.shape

torch.Size([2, 512, 512])

In [8]:
_, predict1 = torch.max(mask_pred1.data, 0)
#predict1 = mask_pred1.data[1] > 0
predict1_np = predict1.numpy()
inverted_fault_mask = (255 - predict1_np * 255).astype('uint8')
transform = transforms.Resize((512, 512))
img = Image.open(f"../data/aug_raw_seismic_hasfault/seismic-{code}.png")
img = np.asarray(transform(img))
# Convert binary image to a 3-channel image for overlay (BGR format)
fault_mask_bgr = cv2.cvtColor(inverted_fault_mask, cv2.COLOR_GRAY2BGR)

overlay = cv2.addWeighted(img, 0.5, fault_mask_bgr, 0.5, 0)
print(inverted_fault_mask.mean())
cv2.imwrite("overlay_img_fault1.png", overlay)

242.01187133789062


True

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from scipy.ndimage import gaussian_filter


In [218]:
def wiener_deblur(blurred, kernel, noise_level=1e-12):
    """
    Wiener deconvolution for approximate deblurring.
    
    Parameters:
    blurred (numpy.ndarray): Blurred image.
    kernel (numpy.ndarray): Gaussian blur kernel.
    noise_level (float): Regularization parameter.
    
    Returns:
    numpy.ndarray: Deblurred image approximation.
    """
    # Pad kernel to the size of the image
    # kernel_padded = np.zeros_like(blurred)
    kernel_padded = blurred.detach().numpy()
    kh, kw = kernel.shape
    kernel_padded[:kh, :kw] = kernel
    
    # Fourier transforms
    kernel_ft = np.fft.fft2(kernel_padded)
    blurred_ft = np.fft.fft2(blurred.detach().numpy())
    # blurred_ft = np.fft.fft2(kernel_padded)
    
    # Wiener deconvolution formula
    kernel_ft_conj = np.conj(kernel_ft)
    deblurred_ft = (kernel_ft_conj / (kernel_ft * kernel_ft_conj + noise_level)) * blurred_ft
    
    # Inverse Fourier transform to get back the image
    deblurred = np.fft.ifft2(deblurred_ft).real
    return np.clip(deblurred, 0, 1)

In [219]:
# Parameters
blurred = mask_pred[0][1]
print(blurred.shape)
sigma = 1e4 # Blur intensity
noise_level = 1e-2

# Gaussian blur kernel
kernel_size = 15
kernel = np.zeros((kernel_size, kernel_size))
kernel[kernel_size // 2, kernel_size // 2] = 1
kernel = gaussian_filter(kernel, sigma=sigma)

deblurred = wiener_deblur(blurred, kernel, noise_level=noise_level)

torch.Size([512, 512])


In [220]:
mask_pred.shape

torch.Size([1, 2, 512, 512])

In [145]:
deblurred.mean()

np.float32(1.0340703e-05)

In [146]:
inverted_fault_mask = (255 - deblurred * 255).astype('uint8')
transform = transforms.Resize((512, 512))
img = Image.open(f"../data/aug_raw_seismic_hasfault/seismic-{code}.png")
img = np.asarray(transform(img))
# Convert binary image to a 3-channel image for overlay (BGR format)
fault_mask_bgr = cv2.cvtColor(inverted_fault_mask, cv2.COLOR_GRAY2BGR)
print(fault_mask_bgr)
overlay = cv2.addWeighted(img, 0.5, fault_mask_bgr, 0.5, 0)

cv2.imwrite("overlay_img_fault2.png", overlay)

[[[  0   0   0]
  [254 254 254]
  [254 254 254]
  ...
  [254 254 254]
  [254 254 254]
  [254 254 254]]

 [[254 254 254]
  [254 254 254]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[254 254 254]
  [255 255 255]
  [254 254 254]
  ...
  [254 254 254]
  [254 254 254]
  [255 255 255]]

 ...

 [[254 254 254]
  [255 255 255]
  [254 254 254]
  ...
  [255 255 255]
  [255 255 255]
  [254 254 254]]

 [[254 254 254]
  [255 255 255]
  [254 254 254]
  ...
  [254 254 254]
  [254 254 254]
  [255 255 255]]

 [[254 254 254]
  [255 255 255]
  [255 255 255]
  ...
  [254 254 254]
  [255 255 255]
  [254 254 254]]]


True

In [147]:
inverted_fault_mask.mean()

np.float64(254.4996452331543)

In [169]:
def unsharp_mask(image, sigma=0.001, strength=0.001):
    """
    Apply unsharp masking to an image.
    
    Parameters:
    image (numpy.ndarray): Input image.
    sigma (float): Sigma for Gaussian blur.
    strength (float): Scaling factor for enhancement.
    
    Returns:
    numpy.ndarray: Sharpened image.
    """
    blurred = gaussian_filter(image.detach().numpy(), sigma=sigma)
    mask = image.detach().numpy() - blurred
    sharpened = image.detach().numpy() + strength * mask
    return np.clip(sharpened, 0, 1)  # Assuming normalized [0, 1] input


In [170]:
sharpened_image = unsharp_mask(blurred)

In [171]:
sharpened_image.mean()

np.float32(0.0476022)

In [172]:
inverted_fault_mask = (255 - sharpened_image * 255).astype('uint8')
transform = transforms.Resize((512, 512))
img = Image.open(f"../data/aug_raw_seismic_hasfault/seismic-{code}.png")
img = np.asarray(transform(img))
# Convert binary image to a 3-channel image for overlay (BGR format)
fault_mask_bgr = cv2.cvtColor(inverted_fault_mask, cv2.COLOR_GRAY2BGR)
print(fault_mask_bgr)
overlay = cv2.addWeighted(img, 0.5, fault_mask_bgr, 0.5, 0)

cv2.imwrite("overlay_img_fault3.png", overlay)

[[[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[253 253 253]
  [253 253 253]
  [253 253 253]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [231 231 231]
  [233 233 233]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[242 242 242]
  [254 254 254]
  [252 252 252]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[241 241 241]
  [249 249 249]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]


True

In [177]:
mask_pred[0][1].

tensor([[0.0044, 0.0044, 0.0044,  ..., 0.0000, 0.0000, 0.0000],
        [0.0044, 0.0044, 0.0044,  ..., 0.0000, 0.0000, 0.0000],
        [0.0044, 0.0044, 0.0044,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0920, 0.0854,  ..., 0.0000, 0.0000, 0.0000],
        [0.0505, 0.0002, 0.0087,  ..., 0.0000, 0.0000, 0.0000],
        [0.0537, 0.0197, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<SelectBackward0>)

In [229]:
# inverted_fault_mask = (255 - sharpened_image * 255).astype('uint8')

kernel_size = 3  # Size of the structuring element

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
# sheng_mask = mask_pred[0][1].detach().numpy().astype('uint8')
sheng_mask = cv2.erode(sharpened_image, kernel, iterations=1)
#sheng_mask = (255 - sheng_mask * 255).astype('uint8')
#sheng_mask = cv2.bitwise_not(sheng_mask)
#sheng_mask = cv2.erode(sheng_mask, kernel, iterations=1)
#sheng_mask = cv2.bitwise_not(sheng_mask) # invert back


transform = transforms.Resize((512, 512))
img = Image.open(f"../data/aug_raw_seismic_hasfault/seismic-{code}.png")
img = np.asarray(transform(img))
# # Convert binary image to a 3-channel image for overlay (BGR format)
sheng_mask = cv2.cvtColor(sheng_mask, cv2.COLOR_GRAY2BGR)
# print(img.shape)
# print(sheng_mask.shape)


# # print(fault_mask_bgr)
overlay = cv2.addWeighted(img, 0.5, sheng_mask, 0.5, 0)

cv2.imwrite("overlay_img_fault3.png", overlay)
# cv2.imwrite("overlay_img_fault3.png", sheng_mask)

True